Please ensure you have run all previous notebooks in sequence before running this.

#Model Building

In [3]:
import os
import pprint
import numpy as np

from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [4]:
#get the train and test datasets
train_data_path = "BikeBuyerTrain"
test_data_path = "BikeBuyerTest"

train = spark.read.parquet(train_data_path)
test = spark.read.parquet(test_data_path)

print("train: ({}, {})".format(train.count(), len(train.columns)))
print("test: ({}, {})".format(test.count(), len(test.columns)))

train.printSchema()

train: (3663, 12)
test: (1212, 12)
root
-- MaritalStatus: integer (nullable = true)
-- Gender: integer (nullable = true)
-- YearlyIncome: integer (nullable = true)
-- TotalChildren: integer (nullable = true)
-- NumberChildrenAtHome: integer (nullable = true)
-- Education: integer (nullable = true)
-- HouseOwnerFlag: integer (nullable = true)
-- NumberCarsOwned: integer (nullable = true)
-- CommuteDistance: integer (nullable = true)
-- Region: integer (nullable = true)
-- Age: integer (nullable = true)
-- BikeBuyer: integer (nullable = true)

#Define ML Pipeline

In [6]:
label = "BikeBuyer"

reg = 0.1
print("Regularization Rate is {}.".format(reg))

# create a new Logistic Regression model.
lr = LogisticRegression(regParam=reg)

dtypes = dict(train.dtypes)
dtypes.pop(label)

si_xvars = []
ohe_xvars = []
featureCols = []
for idx,key in enumerate(dtypes):
    if dtypes[key] == "string":
        featureCol = "-".join([key, "encoded"])
        featureCols.append(featureCol)
        
        tmpCol = "-".join([key, "tmp"])
        # string-index and one-hot encode the string column
        #https://spark.apache.org/docs/2.3.0/api/java/org/apache/spark/ml/feature/StringIndexer.html
        #handleInvalid: Param for how to handle invalid data (unseen labels or NULL values). 
        #Options are 'skip' (filter out rows with invalid data), 'error' (throw an error), 
        #or 'keep' (put invalid data in a special additional bucket, at index numLabels). Default: "error"
        si_xvars.append(StringIndexer(inputCol=key, outputCol=tmpCol, handleInvalid="skip")) #, handleInvalid="keep"
        ohe_xvars.append(OneHotEncoder(inputCol=tmpCol, outputCol=featureCol))
    else:
        featureCols.append(key)

# string-index the label column into a column named "label"
si_label = StringIndexer(inputCol=label, outputCol='label')

# assemble the encoded feature columns in to a column named "features"
assembler = VectorAssembler(inputCols=featureCols, outputCol="features")

# put together the pipeline
pipe = Pipeline(stages=[*si_xvars, *ohe_xvars, si_label, assembler, lr])

# train the model
model = pipe.fit(train)
print(model)

Regularization Rate is 0.1.
PipelineModel_890d548c1ee2

#Tune ML Pipeline

In [8]:
regs = np.arange(0.0, 1.0, 0.2)

paramGrid = ParamGridBuilder().addGrid(lr.regParam, regs).build()
cv = CrossValidator(estimator=pipe, evaluator=BinaryClassificationEvaluator(), estimatorParamMaps=paramGrid)

In [9]:
cvModel = cv.fit(train)

In [10]:
model = cvModel.bestModel

#Model Evaluation

In [12]:
# make prediction
pred = model.transform(test)
output = pred[['MaritalStatus','Age','Gender','Region','Education','YearlyIncome','NumberCarsOwned','TotalChildren','NumberChildrenAtHome','HouseOwnerFlag','CommuteDistance','BikeBuyer']]
display(output.limit(25))

MaritalStatus,Age,Gender,Region,Education,YearlyIncome,NumberCarsOwned,TotalChildren,NumberChildrenAtHome,HouseOwnerFlag,CommuteDistance,BikeBuyer
1,39,1,2,1,10000,1,0,0,0,5,1
1,39,1,2,1,10000,2,0,0,0,1,0
1,39,1,2,1,10000,2,0,0,0,1,1
1,39,1,2,1,10000,2,0,0,0,1,1
1,39,1,2,1,10000,2,0,0,0,1,1
1,39,1,2,1,10000,2,0,0,1,1,1
1,37,1,2,1,10000,0,1,1,0,2,1
1,38,1,2,1,10000,0,1,1,0,2,1
1,39,1,2,1,10000,0,1,1,0,2,1
1,39,1,2,1,10000,0,1,1,0,2,1


Note that the predictions and probabilities are not displayed in the dataframe above?  Why is that?  How can we add them to the printed output?  

Now let's look at the ROC and Precision Recall curves.

In [14]:
# evaluate. note only 2 metrics are supported out of the box by Spark ML.
bce = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction')
au_roc = bce.setMetricName('areaUnderROC').evaluate(pred)
au_prc = bce.setMetricName('areaUnderPR').evaluate(pred)

print("Area under ROC: {}".format(au_roc))
print("Area Under PR: {}".format(au_prc))

Area under ROC: 0.6581025086124398
Area Under PR: 0.5482408921823594

#Model Persistence

In [16]:
##NOTE: by default the model is saved to and loaded from /dbfs/ instead of cwd!
model_name = "BikeBuyer.mml"
model_dbfs = os.path.join("/dbfs", model_name)

model.write().overwrite().save(model_name)
print("saved model to {}".format(model_dbfs))

saved model to /dbfs/BikeBuyer.mml

What does an mml formatted model look like?

In [18]:
%sh

ls -la /dbfs/BikeBuyer.mml/*

/dbfs/BikeBuyer.mml/metadata:
total 0
drwxr-xr-x 1 root root 0 Oct 23 18:54 .
drwxr-xr-x 1 root root 0 Oct 23 18:54 ..
-rw-r--r-- 1 root root 272 Oct 23 18:54 part-00000
-rw-r--r-- 1 root root 0 Oct 23 18:54 _SUCCESS

/dbfs/BikeBuyer.mml/stages:
total 0
drwxr-xr-x 1 root root 0 Oct 23 18:54 .
drwxr-xr-x 1 root root 0 Oct 23 18:54 ..
drwxr-xr-x 1 root root 0 Oct 23 18:54 0_StringIndexer_91497bc21607
drwxr-xr-x 1 root root 0 Oct 23 18:54 1_VectorAssembler_0c0af7d66a78
drwxr-xr-x 1 root root 0 Oct 23 18:54 2_LogisticRegression_6bf393da1840

In [19]:
dbutils.notebook.exit("success")

success